# **EXPERIMENT 6**
## **Name**: Rishikesh Vadodaria
## **Roll no**: C114
## **Aim**: Monte Carlo Prediction

**Maze Structure:**
```
S . .
# . #
. . G
```



In [1]:
import numpy as np
import pandas as pd

import time
from collections import defaultdict

# Maze Environment
class MazeEnv:
    def __init__(self):
        self.grid = [
            ['S', '.', '.'],
            ['#', '.', '#'],
            ['.', '.', 'G']
        ]
        self.rows = len(self.grid)
        self.cols = len(self.grid[0])
        self.start_pos = (0, 0)  # Start position
        self.goal_pos = (2, 2)   # Goal position
        self.state = self.start_pos
        self.actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

    def reset(self):
        self.state = self.start_pos
        return self.state

    def step(self, action):
        row, col = self.state
        new_row = row + action[0]
        new_col = col + action[1]

        # Check if the new position is valid
        if 0 <= new_row < self.rows and 0 <= new_col < self.cols:
            if self.grid[new_row][new_col] == '#':  # Hit a wall
                return self.state, -1, False
            elif self.grid[new_row][new_col] == 'G':  # Reach the goal
                self.state = (new_row, new_col)
                return self.state, 1, True
            else:  # Move to a free space
                self.state = (new_row, new_col)
                return self.state, 0, False
        else:  # Out of bounds
            return self.state, -1, False

In [2]:
# First Visit Monte Carlo Prediction
def first_visit_mc_prediction(env, num_episodes, gamma=1.0):
    V = defaultdict(float)  # Value function
    returns = defaultdict(list)  # List of returns for each state

    for episode in range(num_episodes):
        episode_history = []
        state = env.reset()
        done = False

        # Generate an episode
        while not done:
            action = env.actions[np.random.choice(len(env.actions))]  # Random policy
            next_state, reward, done = env.step(action)
            episode_history.append((state, reward))
            state = next_state

        # Calculate returns and update value function
        G = 0
        first_visit = set()
        for t in reversed(range(len(episode_history))):
            state, reward = episode_history[t]
            G = gamma * G + reward
            if state not in first_visit:
                first_visit.add(state)
                returns[state].append(G)
                V[state] = np.mean(returns[state])

    return V

In [3]:
# Every Visit Monte Carlo Prediction
def every_visit_mc_prediction(env, num_episodes, gamma=1.0):
    V = defaultdict(float)  # Value function
    returns = defaultdict(list)  # List of returns for each state

    for episode in range(num_episodes):
        episode_history = []
        state = env.reset()
        done = False

        # Generate an episode
        while not done:
            action = env.actions[np.random.choice(len(env.actions))]  # Random policy
            next_state, reward, done = env.step(action)
            episode_history.append((state, reward))
            state = next_state

        # Calculate returns and update value function
        G = 0
        for t in reversed(range(len(episode_history))):
            state, reward = episode_history[t]
            G = gamma * G + reward
            returns[state].append(G)
            V[state] = np.mean(returns[state])

    return V

In [4]:
# Exploring Starts Monte Carlo Prediction
def exploring_starts_mc_prediction(env, num_episodes, gamma=1.0):
    V = defaultdict(float)  # Value function
    returns = defaultdict(list)  # List of returns for each state

    for episode in range(num_episodes):
        # Start from a random state-action pair
        state = (np.random.randint(env.rows), np.random.randint(env.cols))
        while env.grid[state[0]][state[1]] == '#':  # Ensure the state is not a wall
            state = (np.random.randint(env.rows), np.random.randint(env.cols))
        env.state = state

        episode_history = []
        done = False

        # Generate an episode
        while not done:
            action = env.actions[np.random.choice(len(env.actions))]  # Random policy
            next_state, reward, done = env.step(action)
            episode_history.append((state, reward))
            state = next_state

        # Calculate returns and update value function
        G = 0
        first_visit = set()
        for t in reversed(range(len(episode_history))):
            state, reward = episode_history[t]
            G = gamma * G + reward
            if state not in first_visit:
                first_visit.add(state)
                returns[state].append(G)
                V[state] = np.mean(returns[state])

    return V

In [5]:
# Compare Algorithms
env = MazeEnv()
num_episodes = 1000

# First Visit Monte Carlo
start_time = time.time()
V_first_visit = first_visit_mc_prediction(env, num_episodes)
first_visit_time = time.time() - start_time

# Every Visit Monte Carlo
start_time = time.time()
V_every_visit = every_visit_mc_prediction(env, num_episodes)
every_visit_time = time.time() - start_time

# Exploring Starts Monte Carlo
start_time = time.time()
V_exploring_starts = exploring_starts_mc_prediction(env, num_episodes)
exploring_starts_time = time.time() - start_time

# Create DataFrames for value tables
def create_value_df(V, rows, cols):
    values = np.zeros((rows, cols))
    for row in range(rows):
        for col in range(cols):
            values[row, col] = V[(row, col)]
    return pd.DataFrame(
        values,
        index=[f"Row {i}" for i in range(rows)],
        columns=[f"Col {j}" for j in range(cols)]
    )

df_first_visit = create_value_df(V_first_visit, env.rows, env.cols)
df_every_visit = create_value_df(V_every_visit, env.rows, env.cols)
df_exploring_starts = create_value_df(V_exploring_starts, env.rows, env.cols)

In [6]:
# Print results
print(f"Grid States:")
for row in env.grid:
    print(' '.join(row))
print()

print("Exploring Starts Monte Carlo Value Table:")
display(df_exploring_starts)
print(f"\nTime taken: {exploring_starts_time:.4f} seconds\n")

print("\nFirst Visit Monte Carlo Value Table:")
display(df_first_visit)
print(f"\nTime taken: {first_visit_time:.4f} seconds\n")

print("\nEvery Visit Monte Carlo Value Table:")
display(df_every_visit)
print(f"\nTime taken: {every_visit_time:.4f} seconds\n")

# Compare time complexity
print(f"\n{'-+'*10} Time Complexity Comparison {'-+'*10}")
print(f"Exploring Starts Monte Carlo: {exploring_starts_time:.4f} seconds")
print(f"First Visit Monte Carlo: {first_visit_time:.4f} seconds")
print(f"Every Visit Monte Carlo: {every_visit_time:.4f} seconds")

Grid States:
S . .
# . #
. . G

Exploring Starts Monte Carlo Value Table:


,Col 0,Col 1,Col 2
Row 0,-6.814672,-3.101362,-7.412214
Row 1,0.000000,-1.065891,0.000000
Row 2,-4.453448,1.000000,-13.980519



Time taken: 0.7097 seconds


First Visit Monte Carlo Value Table:


,Col 0,Col 1,Col 2
Row 0,-7.235000,-3.148,-7.45122
Row 1,0.000000,-1.155,0.00000
Row 2,-3.729293,1.000,0.00000



Time taken: 4.2616 seconds


Every Visit Monte Carlo Value Table:


,Col 0,Col 1,Col 2
Row 0,-30.266066,-27.296608,-30.805073
Row 1,0.000000,-20.084862,0.000000
Row 2,-13.550562,-10.922880,0.000000



Time taken: 30.0759 seconds


-+-+-+-+-+-+-+-+-+-+ Time Complexity Comparison -+-+-+-+-+-+-+-+-+-+
Exploring Starts Monte Carlo: 0.7097 seconds
First Visit Monte Carlo: 4.2616 seconds
Every Visit Monte Carlo: 30.0759 seconds
